**interoperability with NumPy**


Using arbitrary objects in NumPy

In [1]:
import numpy as np

In [2]:
x = np.array([1, 2, 5.0, 8])
x.__array_interface__

{'data': (2725586899824, False),
 'strides': None,
 'descr': [('', '<f8')],
 'typestr': '<f8',
 'shape': (4,),
 'version': 3}

In [3]:
class wrapper():
    pass
arr = np.array([1, 2, 3, 4])
buf = arr.__array_interface__
buf

{'data': (2725719005744, False),
 'strides': None,
 'descr': [('', '<i4')],
 'typestr': '<i4',
 'shape': (4,),
 'version': 3}

In [4]:
buf['shape'] = (2, 2)
w = wrapper()
w.__array_interface__ = buf
new_arr = np.array(w, copy=False)
new_arr

array([[1, 2],
       [3, 4]])

In [5]:
new_arr[0, 0] = 1000
new_arr

array([[1000,    2],
       [   3,    4]])

In [6]:
arr

array([1000,    2,    3,    4])

Operating on foreign objects without converting

In [7]:
def f(x):
    return np.mean(np.exp(x))

In [8]:
x = np.array([1, 2, 3, 4])
f(x)

21.1977562209304

Interoperability examples

Pandas Series objects

In [9]:
import pandas as pd
ser = pd.Series([1, 2, 3, 4])
type(ser)

pandas.core.series.Series

In [10]:
np.exp(ser)

0     2.718282
1     7.389056
2    20.085537
3    54.598150
dtype: float64

In [11]:
np.sin(ser)

0    0.841471
1    0.909297
2    0.141120
3   -0.756802
dtype: float64

In [12]:
np.add(ser, np.array([5, 6, 7, 8]))

0     6
1     8
2    10
3    12
dtype: int64

In [13]:
f(ser)

21.1977562209304

In [14]:
result = ser.__array__()
type(result)

numpy.ndarray

PyTorch tensors

In [15]:
import torch
data = [[1, 2],[3, 4]]
x_np = np.array(data)
x_tensor = torch.tensor(data)
x_np

c:\Users\Yepco\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Yepco\AppData\Local\Temp\ipykernel_24448\4040401536.py:4: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  x_tensor = torch.tensor(data)


array([[1, 2],
       [3, 4]])

In [16]:
x_tensor

tensor([[1, 2],
        [3, 4]])

In [17]:
np.exp(x_tensor)

RuntimeError: Numpy is not available

In [8]:
import torch
t = torch.arange(4)
np.abs(t)

RuntimeError: Numpy is not available

CuPy arrays

In [15]:
import numpy as np
import cupy as cp
x_gpu = cp.array([1, 2, 3])

ModuleNotFoundError: No module named 'cupy'

In [14]:
import cupy as cp
x_gpu = cp.array([1, 2, 3, 4])

ModuleNotFoundError: No module named 'cupy'

In [12]:
np.mean(np.exp(x_gpu))

NameError: name 'x_gpu' is not defined

In [13]:
arr = cp.random.randn(1, 2, 3, 4).astype(cp.float32)
result = np.sum(arr)
print(type(result))

NameError: name 'cp' is not defined

In [18]:
a = np.random.randn(100, 100)
a_gpu = cp.asarray(a)
qr_gpu = np.linalg.qr(a_gpu)

NameError: name 'cp' is not defined

Dask arrays

In [19]:
import dask.array as da
x = da.random.normal(1, 0.1, size=(20, 20), chunks=(10, 10))
np.mean(np.exp(x))

ModuleNotFoundError: No module named 'dask'

In [20]:
np.mean(np.exp(x)).compute()

AttributeError: 'numpy.float64' object has no attribute 'compute'

DLPack

In [21]:
import torch
x_torch = torch.arange(5)
x_torch

tensor([0, 1, 2, 3, 4])

In [22]:
x_np = np.from_dlpack(x_torch)
x_np

AttributeError: module 'numpy' has no attribute 'from_dlpack'

In [23]:
x_torch[1] = 100
x_torch

tensor([  0, 100,   2,   3,   4])

In [24]:
x_np

array([[1, 2],
       [3, 4]])

In [25]:
x.flags.writeable

True

In [26]:
x_np[1] = 1

In [27]:
x_np_copy = x_np.copy()
x_np_copy.sort()  # works

In [28]:
x_torch = torch.arange(5, device='cuda')
np.from_dlpack(x_torch)

AssertionError: Torch not compiled with CUDA enabled

In [29]:
x_torch = torch.arange(5, device='cuda')
x_cupy = cupy.from_dlpack(x_torch)

AssertionError: Torch not compiled with CUDA enabled

In [30]:
x_np = np.arange(5)
x_torch = torch.from_dlpack(x_np)

In [31]:
x_np = np.arange(5)
x_np.flags.writeable = False
torch.from_dlpack(x_np)

TypeError: NumPy currently only supports dlpack for writeable arrays